* code by Sihyun You (2021.12.28.)
* edit by Jehyun Lee (2021.12.30.)

In [149]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from pybliometrics.scopus import ScopusSearch
from bs4 import BeautifulSoup

In [42]:
APIKeys = ["967ac6c5db7b8b34c178035e0ea7e22c", 
           "55eef3b02fc28c636e2fada035cf03d2", 
           "e1fc56d1d9fcc86e6998a7fd79faed23", 
           "dcbed1c3abb36a8e80addbf185f38671", 
           "78d8e510bda00c5530f560f02bc051bc", 
           "4b9ca0308c3ae9b650f74fd19d07d657", 
           "8b02da5833a012844657fc27d129349f", 
           "6624310bd1610368c82261bc6d437be4"]

In [2]:
# subscriber
try:
    s_sample = ScopusSearch(f"DOI (10.1038/s41598-021-83315-9)").results[0]
    subscriber=True
except:
    subscriber=False

print(f"subscriber={subscriber}")    

subscriber=True


In [3]:
def regularize_date_publication(_str):
    token_date = _str.split(' ')
    if (re.match('[0-9]', _str[0])):
        day, month, year = token_date[0], token_date[1][:3].upper(), token_date[2]
        s = ' '.join([day, month, year])
    else:
        month, year = token_date[0][:3].upper(), token_date[1]
        s = ' '.join([month, year])
        
    return (int(year), s)

def get_pub_index(_pub_name, _df):
    list_title = _df['TITLE'].map(regularize)
    pub_index = np.where(regularize(_pub_name) == list_title)[0]
    if len(pub_index) > 0:
        return pub_index
    return -1

def regularize(_str):
    return re.sub('[^A-Za-z0-9]+', '', re.sub('&', 'and', _str)).lower()       

In [7]:
pd.set_option("mode.chained_assignment", None)
pd.set_option("display.max_columns", None)
dfs_JCR_SCIE = {}
YEAR_START, YEAR_REMARK, YEAR_THIS = 2016, 2019, 2021

In [8]:
for i in range(YEAR_START, YEAR_THIS):
    print(f"{i}년도 시트를 로딩중입니다.")
    dfs_JCR_SCIE.update({str(i):pd.read_excel(f"./data/JCR_SCIE_{i}.xlsx")})

2016년도 시트를 로딩중입니다.
2017년도 시트를 로딩중입니다.
2018년도 시트를 로딩중입니다.
2019년도 시트를 로딩중입니다.
2020년도 시트를 로딩중입니다.


In [9]:
dfs_JCR_SCIE.keys()

dict_keys(['2016', '2017', '2018', '2019', '2020'])

In [10]:
df_applicants = pd.read_excel("applicants.xlsx", header=1)
df_applicants.head(3)

,순번,지원번호 내 순번,지원번호\nApplication No.,이름(국문),이름 (영문)\nName,논문명 (Title),게재일자\nPublication Date,학술지구분,저자구분,DOIs (Final),SCIE (Y/N),Publication Year,Publication Date,#citation,Publication Year journal impact factor,2019\njournal\nimpact\nfactor,2019 journal impact factor percentile,CNCI,1st Author,1ST AUTHOR\n(Y/N),Reprint Author,REPRINT AUTHOR\n(Y/N),Source\n(Journal),volume,issue
0,1,1,0026-000001,박성일,"Park, Seong-Il",Estimating blue carbon accumulated in a haloph...,2021.05.21,국외SCIE,주저자,10.1007/s11852-021-00828-1,Y,2021.0,JUN 2021,0.0,NaN,1.374,43.457944,0.0,"Park, Seong-Il",Y,"Um, Jung-Sup",N,JOURNAL OF COASTAL CONSERVATION,25.0,3.0
1,2,3,0026-000003,김기덕,"Kim, Ki-Duk",Beneficial Roles of Carbon Black Additives in ...,2020.10.25,국외SCIE,주저자,10.1016/j.apcata.2020.117837,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,7,0026-000009,김효원,"Kim, HyoWon",Stabilizing role of Mo in TiO2-MoOx supported ...,2020.08.14,국외SCIE,기타,10.1016/j.apcatb.2020.119433,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# df_applicants = df_applicants.drop(["CNCI"], axis = 1)
list_name_en = df_applicants["이름 (영문)\nName"].values
list_doi = df_applicants["DOIs (Final)"].values

In [164]:
xmls = []

for i, doi in enumerate(list_doi):
    s = ScopusSearch(f"DOI ({doi})", download=True, subscriber=subscriber).results
    if s == None:
        print(f"- No.{i} is invalid.")
        continue
        
    info_scopus = s[0]

    index_remark = get_pub_index(info_scopus.publicationName, dfs_JCR_SCIE[str(YEAR_REMARK)])

    scie_yn = 'N'
    if len(index_remark) > 0:
        scie_yn = 'Y'
    else:
        print("SCIE 논문이 아닙니다.")
        continue
    
    df_applicants["SCIE (Y/N)"][i] = scie_yn
    
    year, date = regularize_date_publication(info_scopus.coverDisplayDate)
    df_applicants["Publication Year"][i] = year
    df_applicants["Publication Date"][i] = date
    df_applicants["#citation"][i] = str(info_scopus.citedby_count)
    
    if year < YEAR_THIS:
        index_n = get_pub_index(info_scopus.publicationName, dfs_JCR_SCIE[str(year)])[0]
        jif_n = str(dfs_JCR_SCIE[str(year)]["IMPACT_FACTOR"][index_n])
    else:
        jif_n = ""
        
    df_applicants["Publication Year journal impact factor"][i] = jif_n
    df_applicants["2019\njournal\nimpact\nfactor"][i] = str(dfs_JCR_SCIE[str(YEAR_REMARK)]["IMPACT_FACTOR"][index_remark[0]])
    df_applicants["2019 journal impact factor percentile"][i] = str(dfs_JCR_SCIE[str(YEAR_REMARK)].loc[index_remark, "JIF_PERCENTILE"].max())
    
    author_raw = info_scopus.author_names.split(";")[0]
    if regularize(author_raw) == regularize(list_name_en[i]):
        first_author = list_name_en[i]
        first_author_yn = "Y"
    else:
        first_author = author_raw
        first_author_yn = "N"

    df_applicants["1st Author"][i] = first_author
    df_applicants["1ST AUTHOR\n(Y/N)"][i] = first_author_yn
    
    df_applicants["Source\n(Journal)"][i] = info_scopus.publicationName.upper()
    df_applicants["volume"][i] = info_scopus.volume
    
    if info_scopus.issueIdentifier != None:
        issue = info_scopus.issueIdentifier
    else:
        issue = ''
    df_applicants["issue"][i] = issue
    
    # PDF download
    accept = "application/pdf"
    HEADERS = {
        'X-ELS-APIKEY': APIKeys[-1],
        'Accept': accept
    }
    url = f'http://api.elsevier.com/content/article/doi:{doi}?view=FULL'
    with requests.get(url, stream=True, headers=HEADERS) as r:
        if r.status_code == 200: # download supported in Scopus
            if accept == "application/pdf":
                for chunk in r.iter_content(chunk_size=1024*1024):
                    with open(f"./pdf/paper_{i}.pdf", "wb") as f:
                        f.write(chunk)
            elif accept == "text/xml":
                xml = BeautifulSoup(r.content, "html5lib")
                xmls.append(xml)
                with open(f"./xml/xml_{i}.xml", "w") as f:
                    f.write(xml.prettify())

        elif r.status_code == 404: # not supported in Scopus
            print(f"# No.{i} is not available in Scopus. searching directly.")
            url = f"https://doi.org/{doi}"
            r = requests.get(url)
            xml = BeautifulSoup(r.content, "html5lib")
            xmls.append(xml)            
            with open(f"./xml/xml_{i}.xml", "w") as f:
                f.write(xml.prettify())

print(len(xmls))
df_applicants.to_excel("applicants_fill.xlsx", index=False)

# No.0 is not available in Scopus. searching directly.
# No.5 is not available in Scopus. searching directly.
# No.6 is not available in Scopus. searching directly.
# No.8 is not available in Scopus. searching directly.
# No.9 is not available in Scopus. searching directly.
5


In [165]:
s

[Document(eid='2-s2.0-85034636133', doi='10.1021/acsami.7b11938', pii=None, pubmed_id='29064230', title='Development of Dopant-Free Donor-Acceptor-type Hole Transporting Material for Highly Efficient and Stable Perovskite Solar Cells', subtype='ar', subtypeDescription='Article', creator='Heo J.', afid='60025960;60005273', affilname='Korea Institute of Science and Technology;Korea University', affiliation_city='Seoul;Seoul', affiliation_country='South Korea;South Korea', author_count='4', author_names='Heo, Jin Hyuck;Park, Sungmin;Im, Sang Hyuk;Son, Hae Jung', author_ids='55377501500;57210251100;56998164900;36714465400', author_afids='60005273;60005273-60025960;60005273;60025960', coverDate='2017-11-15', coverDisplayDate='15 November 2017', publicationName='ACS Applied Materials and Interfaces', issn='19448244', source_id='19700171101', eIssn='19448252', aggregationType='Journal', volume='9', issueIdentifier='45', article_number=None, pageRange='39511-39518', description='In perovskite 

In [163]:
s = ScopusSearch(f"AUTHOR ({doi})", download=True, subscriber=subscriber)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_json',
 '_mdate',
 '_n',
 '_view',
 'action',
 'get_cache_file_age',
 'get_cache_file_mdate',
 'get_eids',
 'get_key_remaining_quota',
 'get_key_reset_time',
 'get_results_size',
 'integrity',
 'query',
 'results']